In [1]:
import os

if os.path.basename(os.getcwd()) != 'HUST-NLP-Medical-MultiDocument-Summarization-':
    %cd ../../

e:\pyenv\GTCC\KPG-RL\HUST-NLP-Medical-MultiDocument-Summarization-


In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from transformers import LEDForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoTokenizer
import torch
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from tqdm.notebook import tqdm

In [3]:
RANDOM_SEED = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
PATH = './model/centrum'
tokenizer = AutoTokenizer.from_pretrained(PATH)

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained(PATH)

In [6]:
DOC_SEP_ = "<doc-sep>"
docsep_token_id = tokenizer.convert_tokens_to_ids(DOC_SEP_)

In [7]:
class Test_Medical_Dataset(Dataset):
    def __init__(self,tokenizer:AutoTokenizer,test_data,test_label):
        self.data = test_data
        self.label = test_label
        self.tokenizer = tokenizer
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self,id):
        sentence = self.data.at[id,'Abstracts']
        target = self.label.at[id,'Target']
        review_id = self.data.at[id,"ReviewID"]
        encoding = self.tokenizer(sentence, return_tensors='pt', padding='max_length', truncation=True, max_length=4096)
        global_attention_mask = [[1 if y in [tokenizer.cls_token_id, docsep_token_id] else 0 for y in x]
                                                 for x in encoding['input_ids']]
        return {
            'input_ids': encoding['input_ids'].squeeze(0), # Squeeze to remove the extra dimension
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': target,
            'global_attention_mask': torch.tensor(global_attention_mask).squeeze(0),
            'abstracts': sentence,
            'review_id': review_id
        }

In [8]:
cochrane_dev_input = pd.read_csv("datasets/mslr_data/cochrane/dev-inputs.csv")
cochrane_dev_input['Abstract'].fillna("",inplace = True)
cochrane_dev_input = cochrane_dev_input.groupby('ReviewID').apply(lambda group:"".join([f"{row['Title']}{DOC_SEP_}{row['Abstract']}{DOC_SEP_}" for index, row in group.iterrows()])).reset_index(name="Abstracts")
cochrane_dev_input.sort_values(by='ReviewID', inplace=True)
cochrane_dev_input.reset_index(drop=True, inplace=True)
cochrane_dev_target = pd.read_csv("datasets/mslr_data/cochrane/dev-targets.csv")
cochrane_dev_target.sort_values(by='ReviewID', inplace=True)
cochrane_dev_target.reset_index(drop=True, inplace=True)

test_dataset = Test_Medical_Dataset(tokenizer,cochrane_dev_input,cochrane_dev_target)

In [9]:
res = pd.DataFrame()
res["Abstracts"] = None
res["Label"] = None
res["Generate"] = None
res["ReviewID"] = None

In [10]:
model.to(device)

LEDForConditionalGeneration(
  (led): LEDModel(
    (shared): Embedding(50266, 768, padding_idx=1)
    (encoder): LEDEncoder(
      (embed_tokens): Embedding(50266, 768, padding_idx=1)
      (embed_positions): LEDLearnedPositionalEmbedding(16384, 768)
      (layers): ModuleList(
        (0-5): 6 x LEDEncoderLayer(
          (self_attn): LEDEncoderAttention(
            (longformer_self_attn): LEDEncoderSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_global): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): Linear(in_features=768, out_features=768, bias=True)
          )
     

In [ ]:
k = test_dataset[6]
for sentence in k["abstracts"].split("<doc-sep>"):
    print(sentence+"\n")

In [29]:
gen = model.generate(input_ids=k['input_ids'].unsqueeze(0).to(device),attention_mask=k['attention_mask'].unsqueeze(0).to(device),global_attention_mask=k['global_attention_mask'].unsqueeze(0).to(device),max_length=1024, num_beams=3)
generated_sentence = tokenizer.decode(gen[0], skip_special_tokens=True)
print("Generated result:" + generated_sentence)

A comparison of proximal femoral nail antirotation and dynamic hip screw devices in trochanteric fractures.

This prospective, randomized study compared the functional outcome and complications associated with a proximal femoral nail antirotation (PFNA) device with those of a traditional extramedullary device, the dynamic hip screw (DHS), in patients with trochanteric fracture. A total of 121 patients were randomized to the PFNA group (n = 58) or the DHS group (n = 63). Perioperative information and complications were recorded, and assessments of functional outcome were made. The DHS group required a longer operative time and was associated with greater blood loss than the PFNA group. The re-operation rate was lower in the PFNA group compared with the DHS group, especially in patients with unstable fractures, although there was no statistically significant difference in the overall complication rate between the two groups. There were no significant differences in functional outcome bet